# 001 — CatBoost Regression

Regression with CatBoost, Optuna hyperparameter tuning, residual analysis,
SHAP explanations, and structured metrics output.

**Lifecycle stage:** seedling (model-garden)

All code is self-contained in this notebook — no external library imports
from a shared `src/` package.

In [ ]:
# ---------------------------------------------------------------------------
# Papermill parameters  (this cell is tagged "parameters")
# ---------------------------------------------------------------------------

# Data loading
feature_paths: list[str] = []          # local or gs:// URIs; empty → synthetic
target_paths: list[str] = []           # optional separate target files
join_key: str | None = None            # key to join features ↔ targets
feature_cols: list[str] | None = None  # subset of columns; None → all
target_col: str = "target"
entity_id: str | None = "entity_id"    # column for group-based train/test split

# Splitting
test_size: float = 0.2
random_state: int = 42

# Optuna
optuna_n_trials: int = 30
optuna_timeout_s: int | None = None
optimize_metric: str = "rmse"  # "rmse", "mae", "r2", "mape"

# Outputs
metrics_json_path: str = "outputs/metrics/metrics.json"
model_output_path: str = "outputs/models/model.cbm"
executed_notebook_path: str | None = None
plots_dir: str = "outputs/plots"

# SHAP / feature importance
shap_sample_size: int = 1000
enable_shap: bool = True
enable_feature_importance: bool = True

In [ ]:
# ---------------------------------------------------------------------------
# Imports
# ---------------------------------------------------------------------------
import json
import os
import warnings
from datetime import datetime, timezone
from pathlib import Path

# Suppress all warnings (keeps notebook output and rendered HTML clean)
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from catboost import CatBoostRegressor, Pool
from scipy import stats
from sklearn.datasets import make_regression
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
    median_absolute_error,
    max_error,
    r2_score,
)
from sklearn.model_selection import GroupShuffleSplit, KFold, train_test_split

# Ensure output dirs exist
for d in ["outputs/runs", "outputs/plots", "outputs/models", "outputs/metrics"]:
    Path(d).mkdir(parents=True, exist_ok=True)

print(f"Run started at {datetime.now(timezone.utc).isoformat()}")

## 1 — Data Loading

In [ ]:
# ---------------------------------------------------------------------------
# Data loading helpers (all inline, no external modules)
# ---------------------------------------------------------------------------

def _read_file(path: str) -> pl.DataFrame:
    """Read a single parquet or csv file (local or gs://)."""
    p = path.strip()
    if p.endswith(".parquet") or p.endswith(".pq"):
        return pl.read_parquet(p)
    return pl.read_csv(p)


def load_data(
    feature_paths: list[str],
    target_paths: list[str],
    join_key: str | None,
    feature_cols: list[str] | None,
    target_col: str,
    entity_id: str | None,
) -> pl.DataFrame:
    """Load features (and optional targets), returning a single DataFrame."""

    if not feature_paths:
        # Generate synthetic regression dataset with entity_id
        print("No feature_paths provided — generating synthetic dataset.")
        n_samples = 5_000
        X, y = make_regression(
            n_samples=n_samples,
            n_features=20,
            n_informative=12,
            noise=15.0,
            random_state=random_state,
        )
        cols = {f"feat_{i:02d}": X[:, i] for i in range(X.shape[1])}
        cols[target_col] = y
        # Assign ~3 rows per entity on average so group split is meaningful
        if entity_id:
            rng = np.random.RandomState(random_state)
            n_entities = n_samples // 3
            cols[entity_id] = rng.randint(0, n_entities, size=n_samples)
        return pl.DataFrame(cols)

    # Read and concat feature files
    dfs = [_read_file(p) for p in feature_paths]
    df = pl.concat(dfs, how="vertical_relaxed")

    # Optionally join separate target files
    if target_paths:
        tgt_dfs = [_read_file(p) for p in target_paths]
        tgt = pl.concat(tgt_dfs, how="vertical_relaxed")
        if join_key:
            df = df.join(tgt, on=join_key, how="inner")
        else:
            df = pl.concat([df, tgt], how="horizontal")

    # Subset columns if requested
    if feature_cols is not None:
        keep = list(set(feature_cols + [target_col] + ([entity_id] if entity_id else [])))
        df = df.select([c for c in keep if c in df.columns])

    return df


df = load_data(feature_paths, target_paths, join_key, feature_cols, target_col, entity_id)
print(f"Loaded DataFrame: {df.shape[0]:,} rows × {df.shape[1]} cols")
if entity_id and entity_id in df.columns:
    print(f"Unique entities ({entity_id}): {df[entity_id].n_unique():,}")

## 2 — EDA

In [ ]:
# ---------------------------------------------------------------------------
# Schema & nulls
# ---------------------------------------------------------------------------
print("Schema:")
for name, dtype in zip(df.columns, df.dtypes):
    null_ct = df[name].null_count()
    print(f"  {name:30s}  {str(dtype):15s}  nulls={null_ct}")

print(f"\nTarget summary ({target_col}):")
target_series = df[target_col]
print(f"  count:  {target_series.len():,}")
print(f"  mean:   {target_series.mean():.4f}")
print(f"  std:    {target_series.std():.4f}")
print(f"  min:    {target_series.min():.4f}")
print(f"  25%:    {target_series.quantile(0.25):.4f}")
print(f"  50%:    {target_series.quantile(0.50):.4f}")
print(f"  75%:    {target_series.quantile(0.75):.4f}")
print(f"  max:    {target_series.max():.4f}")

In [ ]:
# ---------------------------------------------------------------------------
# Summary stats for numeric columns
# ---------------------------------------------------------------------------
numeric_cols = [c for c in df.columns if df[c].dtype in (pl.Float32, pl.Float64, pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64)]
feature_numeric = [c for c in numeric_cols if c != target_col]

if feature_numeric:
    print(df.select(feature_numeric).describe())

In [ ]:
# ---------------------------------------------------------------------------
# Target distribution
# ---------------------------------------------------------------------------
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Histogram
ax = axes[0]
target_vals = df[target_col].drop_nulls().to_numpy()
ax.hist(target_vals, bins=50, edgecolor="white", color="#2196F3")
ax.axvline(np.mean(target_vals), color="red", linestyle="--", alpha=0.7, label=f"Mean={np.mean(target_vals):.2f}")
ax.axvline(np.median(target_vals), color="orange", linestyle="--", alpha=0.7, label=f"Median={np.median(target_vals):.2f}")
ax.set_xlabel(target_col)
ax.set_ylabel("Count")
ax.set_title(f"Target Distribution: {target_col}")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# Box plot
ax = axes[1]
bp = ax.boxplot(target_vals, vert=True, patch_artist=True)
bp["boxes"][0].set_facecolor("#2196F3")
bp["boxes"][0].set_alpha(0.6)
ax.set_ylabel(target_col)
ax.set_title(f"Target Box Plot: {target_col}")
ax.grid(True, alpha=0.3)

fig.tight_layout()
fig.savefig(f"{plots_dir}/eda_target_distribution.png", dpi=120)
plt.show()
print(f"Saved → {plots_dir}/eda_target_distribution.png")

In [ ]:
# ---------------------------------------------------------------------------
# Histograms (first 8 numeric features)
# ---------------------------------------------------------------------------
plot_cols = feature_numeric[:8]
if plot_cols:
    n = len(plot_cols)
    ncols = min(4, n)
    nrows = (n + ncols - 1) // ncols
    fig, axes = plt.subplots(nrows, ncols, figsize=(4 * ncols, 3 * nrows))
    axes_flat = np.array(axes).flatten() if n > 1 else [axes]
    for i, col in enumerate(plot_cols):
        axes_flat[i].hist(df[col].drop_nulls().to_numpy(), bins=40, edgecolor="white")
        axes_flat[i].set_title(col, fontsize=10)
    for j in range(i + 1, len(axes_flat)):
        axes_flat[j].set_visible(False)
    fig.tight_layout()
    fig.savefig(f"{plots_dir}/eda_histograms.png", dpi=120)
    plt.show()
    print(f"Saved → {plots_dir}/eda_histograms.png")

In [ ]:
# ---------------------------------------------------------------------------
# Correlation heatmap (cap at 20 features to keep runtime sane)
# ---------------------------------------------------------------------------
corr_cols = feature_numeric[:20]
if len(corr_cols) >= 2:
    corr_df = df.select(corr_cols).to_pandas()
    corr = corr_df.corr()
    fig, ax = plt.subplots(figsize=(max(6, len(corr_cols) * 0.6), max(5, len(corr_cols) * 0.5)))
    im = ax.imshow(corr.values, aspect="auto", cmap="RdBu_r", vmin=-1, vmax=1)
    ax.set_xticks(range(len(corr_cols)))
    ax.set_yticks(range(len(corr_cols)))
    ax.set_xticklabels(corr_cols, rotation=90, fontsize=7)
    ax.set_yticklabels(corr_cols, fontsize=7)
    fig.colorbar(im, ax=ax, shrink=0.8)
    ax.set_title("Pairwise Correlation")
    fig.tight_layout()
    fig.savefig(f"{plots_dir}/eda_correlation.png", dpi=120)
    plt.show()
    print(f"Saved → {plots_dir}/eda_correlation.png")

In [ ]:
# ---------------------------------------------------------------------------
# Feature vs Target scatter plots (top 6 by absolute correlation)
# ---------------------------------------------------------------------------
if feature_numeric:
    # Compute correlation of each feature with target
    target_np = df[target_col].drop_nulls().to_numpy()
    feat_corrs = []
    for col in feature_numeric:
        vals = df[col].drop_nulls().to_numpy()
        if len(vals) == len(target_np) and len(vals) > 1:
            corr_val = np.corrcoef(vals, target_np)[0, 1]
            feat_corrs.append((col, abs(corr_val), corr_val))
    feat_corrs.sort(key=lambda x: x[1], reverse=True)
    top_corr_feats = feat_corrs[:6]

    if top_corr_feats:
        n = len(top_corr_feats)
        ncols = min(3, n)
        nrows = (n + ncols - 1) // ncols
        fig, axes = plt.subplots(nrows, ncols, figsize=(5 * ncols, 4 * nrows))
        axes_flat = np.array(axes).flatten() if n > 1 else [axes]

        for i, (col, abs_corr, corr_val) in enumerate(top_corr_feats):
            ax = axes_flat[i]
            x_vals = df[col].to_numpy()
            ax.scatter(x_vals, target_np, alpha=0.15, s=8, color="#2196F3")
            # Trend line
            z = np.polyfit(x_vals, target_np, 1)
            p = np.poly1d(z)
            x_sorted = np.sort(x_vals)
            ax.plot(x_sorted, p(x_sorted), color="red", linewidth=2, alpha=0.7)
            ax.set_xlabel(col)
            ax.set_ylabel(target_col)
            ax.set_title(f"{col} (r={corr_val:.3f})")
            ax.grid(True, alpha=0.3)

        for j in range(i + 1, len(axes_flat)):
            axes_flat[j].set_visible(False)

        fig.suptitle("Top Features vs Target (by correlation)", fontsize=13, y=1.02)
        fig.tight_layout()
        fig.savefig(f"{plots_dir}/eda_feature_vs_target.png", dpi=120, bbox_inches="tight")
        plt.show()
        print(f"Saved → {plots_dir}/eda_feature_vs_target.png")

## 3 — Train / Test Split

In [ ]:
# ---------------------------------------------------------------------------
# Split — group-based on entity_id (no entity leaks between train/test)
# ---------------------------------------------------------------------------
non_feature_cols = {target_col}
if entity_id and entity_id in df.columns:
    non_feature_cols.add(entity_id)

feature_names = [c for c in df.columns if c not in non_feature_cols]
X_all = df.select(feature_names).to_numpy()
y_all = df[target_col].to_numpy().astype(float)

if entity_id and entity_id in df.columns:
    groups = df[entity_id].to_numpy()
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    train_idx, test_idx = next(gss.split(X_all, y_all, groups=groups))
    X_train, X_test = X_all[train_idx], X_all[test_idx]
    y_train, y_test = y_all[train_idx], y_all[test_idx]
    n_train_entities = len(set(groups[train_idx]))
    n_test_entities = len(set(groups[test_idx]))
    assert len(set(groups[train_idx]) & set(groups[test_idx])) == 0, "Entity leak detected!"
    print(f"Group split on '{entity_id}': {n_train_entities:,} train entities, {n_test_entities:,} test entities")
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X_all, y_all, test_size=test_size, random_state=random_state
    )
    print("Random split (no entity_id column found)")

print(f"Train: {X_train.shape[0]:,}  |  Test: {X_test.shape[0]:,}")
print(f"Train target — mean: {y_train.mean():.4f}  std: {y_train.std():.4f}")
print(f"Test target  — mean: {y_test.mean():.4f}  std: {y_test.std():.4f}")

## 4 — Optuna Hyperparameter Tuning

In [ ]:
# ---------------------------------------------------------------------------
# Optuna objective
# ---------------------------------------------------------------------------
import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)


def _score_regression(y_true, y_pred, metric: str) -> float:
    """Return a score for a regression metric (higher is always better)."""
    if metric == "rmse":
        return -np.sqrt(mean_squared_error(y_true, y_pred))
    elif metric == "mae":
        return -mean_absolute_error(y_true, y_pred)
    elif metric == "r2":
        return r2_score(y_true, y_pred)
    elif metric == "mape":
        return -mean_absolute_percentage_error(y_true, y_pred)
    else:
        raise ValueError(f"Unknown metric: {metric}")


def objective(trial: optuna.Trial) -> float:
    params = {
        "iterations": trial.suggest_int("iterations", 200, 1500),
        "depth": trial.suggest_int("depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-2, 10.0, log=True),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "random_strength": trial.suggest_float("random_strength", 1e-3, 10.0, log=True),
    }

    kf = KFold(n_splits=3, shuffle=True, random_state=random_state)
    fold_scores = []

    for train_idx, val_idx in kf.split(X_train):
        xtr, xvl = X_train[train_idx], X_train[val_idx]
        ytr, yvl = y_train[train_idx], y_train[val_idx]

        model = CatBoostRegressor(
            **params,
            eval_metric="RMSE",
            random_seed=random_state,
            verbose=0,
            early_stopping_rounds=50,
        )
        model.fit(xtr, ytr, eval_set=(xvl, yvl), verbose=0)

        preds = model.predict(xvl)
        score = _score_regression(yvl, preds, optimize_metric)
        fold_scores.append(score)

    return float(np.mean(fold_scores))


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=optuna_n_trials, timeout=optuna_timeout_s)

best_params = study.best_params
print(f"Best CV {optimize_metric}: {study.best_value:.4f}")
print(f"Best params: {json.dumps(best_params, indent=2)}")

## 5 — Train Final Model

In [ ]:
# ---------------------------------------------------------------------------
# Final model on full train set
# ---------------------------------------------------------------------------
final_model = CatBoostRegressor(
    **best_params,
    eval_metric="RMSE",
    random_seed=random_state,
    verbose=100,
)
final_model.fit(X_train, y_train)

# Save model
Path(model_output_path).parent.mkdir(parents=True, exist_ok=True)
final_model.save_model(model_output_path)
print(f"\nModel saved → {model_output_path}")

## 6 — Evaluation

In [ ]:
# ---------------------------------------------------------------------------
# Compute predictions and metrics on test set
# ---------------------------------------------------------------------------
y_pred = final_model.predict(X_test)
residuals = y_test - y_pred

# Core metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)
max_err = max_error(y_test, y_pred)

# Additional stats
explained_variance = 1 - np.var(residuals) / np.var(y_test)
mean_residual = np.mean(residuals)
std_residual = np.std(residuals)

print("Test Set Metrics")
print("=" * 40)
print(f"  RMSE:                {rmse:,.4f}")
print(f"  MAE:                 {mae:,.4f}")
print(f"  R²:                  {r2:.4f}")
print(f"  MAPE:                {mape:.4f}")
print(f"  Median AE:           {medae:,.4f}")
print(f"  Max Error:           {max_err:,.4f}")
print(f"  Explained Variance:  {explained_variance:.4f}")
print(f"  Mean Residual:       {mean_residual:,.4f}")
print(f"  Std Residual:        {std_residual:,.4f}")

In [ ]:
# ---------------------------------------------------------------------------
# Actual vs Predicted scatter plot
# ---------------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(7, 7))
ax.scatter(y_test, y_pred, alpha=0.2, s=12, color="#2196F3", edgecolor="none")

# Perfect prediction line
lims = [
    min(y_test.min(), y_pred.min()),
    max(y_test.max(), y_pred.max()),
]
margin = (lims[1] - lims[0]) * 0.05
lims = [lims[0] - margin, lims[1] + margin]
ax.plot(lims, lims, "k--", alpha=0.5, linewidth=1.5, label="Perfect prediction")

# Best fit line
z = np.polyfit(y_test, y_pred, 1)
p = np.poly1d(z)
x_sorted = np.sort(y_test)
ax.plot(x_sorted, p(x_sorted), color="red", linewidth=2, alpha=0.7, label=f"Fit: y={z[0]:.3f}x+{z[1]:.1f}")

ax.set_xlabel("Actual")
ax.set_ylabel("Predicted")
ax.set_title(f"Actual vs Predicted (R²={r2:.4f})")
ax.legend(loc="upper left")
ax.grid(True, alpha=0.3)
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.set_aspect("equal")
fig.tight_layout()
fig.savefig(f"{plots_dir}/actual_vs_predicted.png", dpi=120)
plt.show()
print(f"Saved → {plots_dir}/actual_vs_predicted.png")

In [ ]:
# ---------------------------------------------------------------------------
# Residuals vs Predicted
# ---------------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(9, 5))
ax.scatter(y_pred, residuals, alpha=0.2, s=12, color="#4CAF50", edgecolor="none")
ax.axhline(0, color="red", linestyle="--", linewidth=1.5, alpha=0.7)

# Add ±1 std bands
ax.axhline(std_residual, color="orange", linestyle=":", alpha=0.5, label=f"+1 std ({std_residual:,.2f})")
ax.axhline(-std_residual, color="orange", linestyle=":", alpha=0.5, label=f"-1 std ({-std_residual:,.2f})")

# LOWESS-style smoothed trend (binned means)
n_bins = 30
pred_sorted_idx = np.argsort(y_pred)
bin_size = len(y_pred) // n_bins
if bin_size > 0:
    bin_centers = []
    bin_means = []
    for b in range(n_bins):
        start = b * bin_size
        end = start + bin_size if b < n_bins - 1 else len(y_pred)
        idx = pred_sorted_idx[start:end]
        bin_centers.append(np.mean(y_pred[idx]))
        bin_means.append(np.mean(residuals[idx]))
    ax.plot(bin_centers, bin_means, color="darkblue", linewidth=2, alpha=0.8, label="Binned mean")

ax.set_xlabel("Predicted")
ax.set_ylabel("Residual (Actual - Predicted)")
ax.set_title("Residuals vs Predicted")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(f"{plots_dir}/residuals_vs_predicted.png", dpi=120)
plt.show()
print(f"Saved → {plots_dir}/residuals_vs_predicted.png")

In [ ]:
# ---------------------------------------------------------------------------
# Residual Distribution
# ---------------------------------------------------------------------------
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram with normal fit
ax = axes[0]
ax.hist(residuals, bins=50, density=True, edgecolor="white", color="#2196F3", alpha=0.7, label="Residuals")
# Overlay normal distribution
x_range = np.linspace(residuals.min(), residuals.max(), 200)
ax.plot(x_range, stats.norm.pdf(x_range, mean_residual, std_residual),
        color="red", linewidth=2, label=f"Normal(μ={mean_residual:.2f}, σ={std_residual:.2f})")
ax.set_xlabel("Residual")
ax.set_ylabel("Density")
ax.set_title("Residual Distribution")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# Q-Q Plot
ax = axes[1]
standardized_residuals = (residuals - mean_residual) / std_residual
theoretical_quantiles = stats.norm.ppf(np.linspace(0.001, 0.999, len(standardized_residuals)))
sorted_residuals = np.sort(standardized_residuals)
ax.scatter(theoretical_quantiles, sorted_residuals, alpha=0.3, s=8, color="#4CAF50")
qq_lims = [min(theoretical_quantiles.min(), sorted_residuals.min()),
           max(theoretical_quantiles.max(), sorted_residuals.max())]
ax.plot(qq_lims, qq_lims, "r--", linewidth=1.5, alpha=0.7, label="Normal reference")
ax.set_xlabel("Theoretical Quantiles")
ax.set_ylabel("Standardized Residuals")
ax.set_title("Q-Q Plot (Residuals vs Normal)")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

fig.tight_layout()
fig.savefig(f"{plots_dir}/residual_distribution.png", dpi=120)
plt.show()
print(f"Saved → {plots_dir}/residual_distribution.png")

In [ ]:
# ---------------------------------------------------------------------------
# Prediction Error Plot (sorted index)
# ---------------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(10, 5))

sort_idx = np.argsort(y_test)
y_test_sorted = y_test[sort_idx]
y_pred_sorted = y_pred[sort_idx]
x_axis = np.arange(len(y_test_sorted))

ax.plot(x_axis, y_test_sorted, color="#2196F3", linewidth=1, alpha=0.8, label="Actual")
ax.plot(x_axis, y_pred_sorted, color="#FF5722", linewidth=1, alpha=0.6, label="Predicted")
ax.fill_between(x_axis, y_test_sorted, y_pred_sorted, alpha=0.15, color="red", label="Error")

ax.set_xlabel("Sample (sorted by actual value)")
ax.set_ylabel("Value")
ax.set_title("Prediction Error: Actual vs Predicted (sorted)")
ax.legend()
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(f"{plots_dir}/prediction_error_sorted.png", dpi=120)
plt.show()
print(f"Saved → {plots_dir}/prediction_error_sorted.png")

In [ ]:
# ---------------------------------------------------------------------------
# Cumulative Error Distribution
# ---------------------------------------------------------------------------
abs_errors = np.abs(residuals)
sorted_errors = np.sort(abs_errors)
cumulative_pct = np.arange(1, len(sorted_errors) + 1) / len(sorted_errors) * 100

fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(sorted_errors, cumulative_pct, linewidth=2, color="#9C27B0")

# Mark key percentiles
for pct in [50, 75, 90, 95]:
    idx = int(len(sorted_errors) * pct / 100) - 1
    err_at_pct = sorted_errors[idx]
    ax.axhline(pct, color="grey", linestyle=":", alpha=0.3)
    ax.axvline(err_at_pct, color="grey", linestyle=":", alpha=0.3)
    ax.plot(err_at_pct, pct, "ro", markersize=6)
    ax.annotate(f"  {pct}%: err≤{err_at_pct:,.2f}",
                xy=(err_at_pct, pct), fontsize=8, va="bottom")

ax.set_xlabel("Absolute Error")
ax.set_ylabel("Cumulative % of Predictions")
ax.set_title("Cumulative Error Distribution")
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(f"{plots_dir}/cumulative_error_distribution.png", dpi=120)
plt.show()
print(f"Saved → {plots_dir}/cumulative_error_distribution.png")

In [ ]:
# ---------------------------------------------------------------------------
# Error by Decile of Actual Value
# ---------------------------------------------------------------------------
n_deciles = 10
decile_edges = np.percentile(y_test, np.linspace(0, 100, n_deciles + 1))
decile_labels = []
decile_mae = []
decile_rmse = []
decile_counts = []

for i in range(n_deciles):
    lo, hi = decile_edges[i], decile_edges[i + 1]
    if i < n_deciles - 1:
        mask = (y_test >= lo) & (y_test < hi)
    else:
        mask = (y_test >= lo) & (y_test <= hi)
    if mask.sum() == 0:
        continue
    decile_labels.append(f"D{i+1}\n[{lo:,.0f},{hi:,.0f})")
    decile_mae.append(mean_absolute_error(y_test[mask], y_pred[mask]))
    decile_rmse.append(np.sqrt(mean_squared_error(y_test[mask], y_pred[mask])))
    decile_counts.append(int(mask.sum()))

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

x = np.arange(len(decile_labels))
width = 0.35

# MAE & RMSE by decile
ax = axes[0]
bars1 = ax.bar(x - width/2, decile_mae, width, label="MAE", color="#2196F3", alpha=0.8)
bars2 = ax.bar(x + width/2, decile_rmse, width, label="RMSE", color="#FF9800", alpha=0.8)
ax.set_xticks(x)
ax.set_xticklabels(decile_labels, fontsize=7)
ax.set_ylabel("Error")
ax.set_title("MAE & RMSE by Decile of Actual Value")
ax.legend()
ax.grid(True, alpha=0.3, axis="y")

# Sample counts by decile
ax = axes[1]
ax.bar(x, decile_counts, color="#4CAF50", alpha=0.8)
ax.set_xticks(x)
ax.set_xticklabels(decile_labels, fontsize=7)
ax.set_ylabel("Count")
ax.set_title("Sample Count by Decile")
ax.grid(True, alpha=0.3, axis="y")

fig.tight_layout()
fig.savefig(f"{plots_dir}/error_by_decile.png", dpi=120)
plt.show()
print(f"Saved → {plots_dir}/error_by_decile.png")

In [ ]:
# ---------------------------------------------------------------------------
# Residuals vs Top Features
# ---------------------------------------------------------------------------
importances_quick = final_model.get_feature_importance()
imp_order = np.argsort(-importances_quick)
top_feat_indices = imp_order[:4]

fig, axes = plt.subplots(1, len(top_feat_indices), figsize=(5 * len(top_feat_indices), 4))
if len(top_feat_indices) == 1:
    axes = [axes]

for ax, fi in zip(axes, top_feat_indices):
    feat_vals = X_test[:, fi]
    ax.scatter(feat_vals, residuals, alpha=0.15, s=8, color="#FF5722")
    ax.axhline(0, color="red", linestyle="--", linewidth=1, alpha=0.5)

    # Binned mean trend
    n_bins = 20
    sorted_idx = np.argsort(feat_vals)
    bin_size = len(feat_vals) // n_bins
    if bin_size > 0:
        bx, by = [], []
        for b in range(n_bins):
            s = b * bin_size
            e = s + bin_size if b < n_bins - 1 else len(feat_vals)
            idx = sorted_idx[s:e]
            bx.append(np.mean(feat_vals[idx]))
            by.append(np.mean(residuals[idx]))
        ax.plot(bx, by, color="darkblue", linewidth=2, alpha=0.8)

    ax.set_xlabel(feature_names[fi])
    ax.set_ylabel("Residual")
    ax.set_title(f"Residual vs {feature_names[fi]}")
    ax.grid(True, alpha=0.3)

fig.tight_layout()
fig.savefig(f"{plots_dir}/residuals_vs_features.png", dpi=120)
plt.show()
print(f"Saved → {plots_dir}/residuals_vs_features.png")

In [ ]:
# ---------------------------------------------------------------------------
# Scale-Location Plot (√|standardized residuals| vs predicted)
# ---------------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(9, 5))

sqrt_abs_std_resid = np.sqrt(np.abs(standardized_residuals))
ax.scatter(y_pred, sqrt_abs_std_resid, alpha=0.2, s=12, color="#FF9800", edgecolor="none")

# Binned mean trend
n_bins = 30
pred_sorted_idx = np.argsort(y_pred)
bin_size = len(y_pred) // n_bins
if bin_size > 0:
    bx, by = [], []
    for b in range(n_bins):
        s = b * bin_size
        e = s + bin_size if b < n_bins - 1 else len(y_pred)
        idx = pred_sorted_idx[s:e]
        bx.append(np.mean(y_pred[idx]))
        by.append(np.mean(sqrt_abs_std_resid[idx]))
    ax.plot(bx, by, color="red", linewidth=2, alpha=0.8, label="Binned mean")

ax.set_xlabel("Predicted")
ax.set_ylabel("√|Standardized Residual|")
ax.set_title("Scale-Location Plot (Homoscedasticity Check)")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(f"{plots_dir}/scale_location.png", dpi=120)
plt.show()
print(f"Saved → {plots_dir}/scale_location.png")

In [ ]:
# ---------------------------------------------------------------------------
# Metrics Dashboard — compact summary of all key metrics
# ---------------------------------------------------------------------------
fig, axes = plt.subplots(2, 3, figsize=(16, 9))

metric_names = ["RMSE", "MAE", "R²", "MAPE", "Median AE", "Max Error"]
metric_values = [rmse, mae, r2, mape, medae, max_err]
metric_colors = ["#2196F3", "#4CAF50", "#FF9800", "#9C27B0", "#00BCD4", "#F44336"]
metric_formats = ["{:,.4f}", "{:,.4f}", "{:.4f}", "{:.4f}", "{:,.4f}", "{:,.4f}"]

for ax, name, val, color, fmt in zip(axes.flat, metric_names, metric_values, metric_colors, metric_formats):
    ax.barh([name], [val], color=color, alpha=0.8, height=0.5)
    ax.text(val, 0, "  " + fmt.format(val), va="center", fontsize=16, fontweight="bold")
    ax.set_title(name, fontsize=14)
    ax.set_xlim(0, val * 1.4 if val > 0 else 1)
    ax.grid(True, alpha=0.3, axis="x")
    ax.set_yticks([])

fig.suptitle("Regression Metrics Dashboard", fontsize=16, y=1.01)
fig.tight_layout()
fig.savefig(f"{plots_dir}/metrics_dashboard.png", dpi=120, bbox_inches="tight")
plt.show()
print(f"Saved → {plots_dir}/metrics_dashboard.png")

In [ ]:
# ---------------------------------------------------------------------------
# Error Percentile Table
# ---------------------------------------------------------------------------
percentiles = [10, 25, 50, 75, 90, 95, 99]
pct_values = np.percentile(abs_errors, percentiles)

pct_rows = [{"percentile": p, "abs_error_at_pct": round(float(v), 4)} for p, v in zip(percentiles, pct_values)]
pct_df = pl.DataFrame(pct_rows)
print("Absolute Error Percentiles:")
print(pct_df)

## 7 — Metrics JSON Output

In [ ]:
# ---------------------------------------------------------------------------
# Write structured metrics JSON
# ---------------------------------------------------------------------------
metrics_output = {
    "run_metadata": {
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "target_col": target_col,
        "entity_id": entity_id,
        "test_size": test_size,
        "random_state": random_state,
        "optuna_n_trials": optuna_n_trials,
        "optimize_metric": optimize_metric,
        "n_train": int(X_train.shape[0]),
        "n_test": int(X_test.shape[0]),
        "n_features": int(X_train.shape[1]),
        "feature_names": feature_names,
        "best_optuna_cv_score": round(study.best_value, 4),
        "best_params": best_params,
        "model_output_path": model_output_path,
    },
    "test_metrics": {
        "rmse": round(rmse, 4),
        "mae": round(mae, 4),
        "r2": round(r2, 4),
        "mape": round(mape, 4),
        "median_ae": round(medae, 4),
        "max_error": round(max_err, 4),
        "explained_variance": round(explained_variance, 4),
        "mse": round(mse, 4),
    },
    "residual_stats": {
        "mean": round(float(mean_residual), 4),
        "std": round(float(std_residual), 4),
        "min": round(float(residuals.min()), 4),
        "max": round(float(residuals.max()), 4),
        "skewness": round(float(stats.skew(residuals)), 4),
        "kurtosis": round(float(stats.kurtosis(residuals)), 4),
    },
    "error_percentiles": {str(p): round(float(v), 4) for p, v in zip(percentiles, pct_values)},
    "target_stats": {
        "train_mean": round(float(y_train.mean()), 4),
        "train_std": round(float(y_train.std()), 4),
        "test_mean": round(float(y_test.mean()), 4),
        "test_std": round(float(y_test.std()), 4),
    },
}

Path(metrics_json_path).parent.mkdir(parents=True, exist_ok=True)
with open(metrics_json_path, "w") as f:
    json.dump(metrics_output, f, indent=2, default=str)

print(f"Metrics JSON saved → {metrics_json_path}")

## 8 — Feature Importance

In [ ]:
# ---------------------------------------------------------------------------
# CatBoost feature importance
# ---------------------------------------------------------------------------
if enable_feature_importance:
    importances = final_model.get_feature_importance()
    imp_df = (
        pl.DataFrame({"feature": feature_names, "importance": importances})
        .sort("importance", descending=True)
    )
    top_n = min(20, len(imp_df))
    top = imp_df.head(top_n)

    fig, ax = plt.subplots(figsize=(8, max(4, top_n * 0.35)))
    ax.barh(top["feature"].to_list()[::-1], top["importance"].to_list()[::-1])
    ax.set_xlabel("Importance")
    ax.set_title(f"Top {top_n} Feature Importances")
    fig.tight_layout()
    fig.savefig(f"{plots_dir}/feature_importance.png", dpi=120)
    plt.show()
    print(f"Saved → {plots_dir}/feature_importance.png")
else:
    print("Feature importance disabled.")

## 9 — SHAP

In [ ]:
# ---------------------------------------------------------------------------
# SHAP explanations
# ---------------------------------------------------------------------------
if enable_shap:
    import shap

    n_sample = min(shap_sample_size, X_test.shape[0])
    rng = np.random.RandomState(random_state)
    idx = rng.choice(X_test.shape[0], size=n_sample, replace=False)
    X_shap = X_test[idx]

    explainer = shap.TreeExplainer(final_model)
    shap_values = explainer.shap_values(X_shap)

    # Summary bar plot
    fig, ax = plt.subplots(figsize=(8, 6))
    shap.summary_plot(
        shap_values, X_shap,
        feature_names=feature_names,
        plot_type="bar",
        show=False,
    )
    plt.tight_layout()
    plt.savefig(f"{plots_dir}/shap_bar.png", dpi=120, bbox_inches="tight")
    plt.show()
    print(f"Saved → {plots_dir}/shap_bar.png")

    # Summary dot plot
    fig, ax = plt.subplots(figsize=(8, 6))
    shap.summary_plot(
        shap_values, X_shap,
        feature_names=feature_names,
        show=False,
    )
    plt.tight_layout()
    plt.savefig(f"{plots_dir}/shap_summary.png", dpi=120, bbox_inches="tight")
    plt.show()
    print(f"Saved → {plots_dir}/shap_summary.png")
else:
    print("SHAP disabled.")

## 10 — Summary

In [ ]:
# ---------------------------------------------------------------------------
# Final summary
# ---------------------------------------------------------------------------
print("=" * 60)
print("RUN COMPLETE")
print("=" * 60)
print(f"  Model saved to:       {model_output_path}")
print(f"  Metrics JSON:         {metrics_json_path}")
print(f"  Plots directory:      {plots_dir}")
if executed_notebook_path:
    print(f"  Executed notebook:    {executed_notebook_path}")
print(f"  RMSE:                 {rmse:,.4f}")
print(f"  MAE:                  {mae:,.4f}")
print(f"  R²:                   {r2:.4f}")
print(f"  MAPE:                 {mape:.4f}")
print(f"  Median AE:            {medae:,.4f}")
print("=" * 60)